In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import keras
import datetime
import plotly.express as px
import ipywidgets as widgets

In [2]:
def read_data(data='Smog.xlsx'):
  
  smog = pd.read_excel(data)
  smog.drop(columns = 'Unnamed: 0',inplace = True)

  return smog

In [3]:
def get_dates(pick_start_date, pick_end_date):

  start_date = pd.Timestamp(pick_start_date)
  end_date = pd.Timestamp(pick_end_date)

  end_date = end_date.replace(hour=23, minute=0, second=0)
  
  if start_date.day == 1:
    start_date = start_date.replace(hour=1, minute=0, second=0)

  return start_date, end_date

In [4]:
def get_date_range(start_date, end_date, data):

    start_index = data.index[data.Data == start_date]
    end_index = data.index[data.Data == end_date]

    days_to_predict = end_date - start_date
    days_to_predict = days_to_predict.days
    
    data.set_index('Data', inplace=True)

    return data, days_to_predict, start_index, end_index

In [5]:
def prepare_data_to(data, scaler, days_to_predict, start_index, end_index, TimeseriesGenerator):

  scaler = MinMaxScaler()
  smog_scaled = scaler.fit_transform(data)
  num_hours = 24 * days_to_predict
  smog_input = smog_scaled[start_index[0] - (num_hours + 120) : start_index[0]]

  win_length = 120    #ostatnie 5 dni
  batch_size = 32     #32 serie do uczenia
  num_features = 11   #liczba features do uczenia
  
  predict_generator = TimeseriesGenerator(smog_input, 
                                        smog_input, 
                                        length=win_length, 
                                        sampling_rate=1, 
                                        batch_size=batch_size)

  return predict_generator, scaler, num_hours

In [6]:
def make_predictions(model, predict_generator, scaler):

  model = keras.models.load_model(model)
  predictions = model.predict(predict_generator)
  smog_out = pd.DataFrame(predictions)
  smog_out_rev = scaler.inverse_transform(smog_out)
  
  return smog_out_rev

In [7]:
def prepare_data_to_plot(data, smog_out_rev, start_date, num_hours):
  
  date_list = [start_date + datetime.timedelta(hours=x) for x in range(num_hours)]
  predictions_df = pd.DataFrame(smog_out_rev)
  predictions_df['Data'] = date_list
  predictions_df.set_index('Data', inplace=True)
  predictions_df.columns = list(data.columns)

  return predictions_df

In [8]:
def make_plot(predictions, widget_one_value, widget_one_options):
  dictionary = dict(widget_one.options)
  title = [k for k, v in dictionary.items() if v == widget_one.value]

  ranges = {'PM10': [20, 50, 80, 110, 150],
            'Dwutlenek azotu': [40, 100, 150, 230, 400]}

  fig = px.line(predictions, 
                x=predictions.index, 
                y=widget_one.value, 
                title=f'Predykowany parametr: {title[0]}')

  if widget_one.value in ranges:

    fig.add_shape(type='line',
                  x0=predictions.index[0],
                  y0=ranges[widget_one.value][0],
                  x1=predictions.index[-1],
                  y1=ranges[widget_one.value][0],
                  line=dict(color='Green'),
                  xref='x',
                  yref='y')
    
    fig.add_shape(type='line',
                x0=predictions.index[0],
                y0=ranges[widget_one.value][1],
                x1=predictions.index[-1],
                y1=ranges[widget_one.value][1],
                line=dict(color='greenyellow'),
                xref='x',
                yref='y')
      
    fig.add_shape(type='line',
                x0=predictions.index[0],
                y0=ranges[widget_one.value][2],
                x1=predictions.index[-1],
                y1=ranges[widget_one.value][2],
                line=dict(color='yellow'),
                xref='x',
                yref='y')
      
    fig.add_shape(type='line',
                x0=predictions.index[0],
                y0=ranges[widget_one.value][3],
                x1=predictions.index[-1],
                y1=ranges[widget_one.value][3],
                line=dict(color='orange'),
                xref='x',
                yref='y')

    fig.add_shape(type='line',
                x0=predictions.index[0],
                y0=ranges[widget_one.value][4],
                x1=predictions.index[-1],
                y1=ranges[widget_one.value][4],
                line=dict(color='red'),
                xref='x',
                yref='y')    

  fig.show()

In [9]:
def show_plot():
  smog = read_data()
  start_date, end_date = get_dates(pick_start_date.value, pick_end_date.value)
  smog, days_to_predict, start_index, end_index = get_date_range(start_date, end_date, smog)
  predict_generator, scaler, num_hours = prepare_data_to(smog, MinMaxScaler, days_to_predict, start_index, end_index, TimeseriesGenerator)
  smog_out_rev = make_predictions('LSTM.h5', predict_generator, scaler)
  predictions = prepare_data_to_plot(smog, smog_out_rev, start_date, num_hours)
  make_plot(predictions, widget_one.value, widget_one.options)

In [10]:
widget_one = widgets.Dropdown(
    options=[('Dwutlenek azotu', 'Dwutlenek_azotu'), 
         ('Tlenki azotu', 'Tlenki_azotu'), 
         ('PM 10', 'PM10'),
         ('PM 2.5', 'PM_2_5'), 
         ('Benzen', 'Benzen'), 
         ('Kierunek wiatru', 'Kierunek_wiatru'),
         ('Prędkość wiatru', 'Predkosc_wiatru'), 
         ('Temperatura', 'Temperatura'), 
         ('Wilgotność', 'Wilgotnosc'),
         ('Ciśnienie', 'Cisnienie')],
    value= 'PM10',
    description='Parametr',
    disabled=False)

pick_start_date = widgets.DatePicker(
    description='Wybierz datę początkową:',
    disabled=False)

pick_end_date = widgets.DatePicker(
    description='Wybierz datę końcową:',
    disabled=False)

In [11]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()


@output.capture()
def on_button_clicked(b):
    show_plot()
    b.icon="warning"

In [12]:
widget_one

Dropdown(description='Parametr', index=2, options=(('Dwutlenek azotu', 'Dwutlenek_azotu'), ('Tlenki azotu', 'T…

In [13]:
pick_start_date

DatePicker(value=None, description='Wybierz datę początkową:')

In [14]:
pick_end_date

DatePicker(value=None, description='Wybierz datę końcową:')

In [15]:
display(button, output)
button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()